In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler


In [5]:
file_path = '/content/dataset.csv'  # Update this path if needed
heart_data = pd.read_csv(file_path)

In [6]:
# Handling missing values (if any)
heart_data.fillna(heart_data.median(), inplace=True)

In [8]:
# Selecting features and target
features = heart_data.drop(columns=['target'])
target = heart_data['target']

# Standardizing numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, target.values, test_size=0.2, random_state=42)

# Converting labels to one-hot encoded format
encoder = OneHotEncoder(handle_unknown='ignore')
y_train_one_hot = encoder.fit_transform(y_train.reshape(-1, 1)).toarray()
y_test_one_hot = encoder.transform(y_test.reshape(-1, 1)).toarray()

In [9]:
# Neural Network Parameters
input_size = X_train.shape[1]  # Number of features
hidden_size = 50  # Hidden layer neurons
output_size = y_train_one_hot.shape[1]  # Output classes

# Initialize weights and biases
np.random.seed(42)
W1 = np.random.randn(input_size, hidden_size) * 0.01
b1 = np.zeros((1, hidden_size))
W2 = np.random.randn(hidden_size, output_size) * 0.01
b2 = np.zeros((1, output_size))

In [10]:
# Activation functions and derivatives
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def cross_entropy_loss(y_pred, y_true):
    m = y_true.shape[0]
    return -np.sum(y_true * np.log(y_pred + 1e-8)) / m

In [11]:
# Forward Pass
def forward(X):
    z1 = np.dot(X, W1) + b1
    a1 = relu(z1)
    z2 = np.dot(a1, W2) + b2
    a2 = softmax(z2)
    return a1, a2

# Backward Pass
def backward(X, y, a1, a2):
    m = X.shape[0]
    dz2 = a2 - y
    dW2 = np.dot(a1.T, dz2) / m
    db2 = np.sum(dz2, axis=0, keepdims=True) / m
    dz1 = np.dot(dz2, W2.T) * relu_derivative(a1)
    dW1 = np.dot(X.T, dz1) / m
    db1 = np.sum(dz1, axis=0, keepdims=True) / m
    return dW1, db1, dW2, db2

In [12]:
# Training function
def train(X_train, y_train, learning_rate=0.01, epochs=1000):
    global W1, b1, W2, b2
    for epoch in range(epochs):
        a1, a2 = forward(X_train)
        loss = cross_entropy_loss(a2, y_train)
        dW1, db1, dW2, db2 = backward(X_train, y_train, a1, a2)
        W1 -= learning_rate * dW1
        b1 -= learning_rate * db1
        W2 -= learning_rate * dW2
        b2 -= learning_rate * db2
        if epoch % 100 == 0:
            print(f"Epoch {epoch}/{epochs}, Loss: {loss:.4f}")

# Prediction function
def predict(X):
    _, a2 = forward(X)
    return np.argmax(a2, axis=1)


In [13]:
# Train the model
train(X_train, y_train_one_hot, learning_rate=0.1, epochs=1000)

# Evaluate the model
y_pred_train = predict(X_train)
y_pred_test = predict(X_test)

y_true_train = np.argmax(y_train_one_hot, axis=1)
y_true_test = np.argmax(y_test_one_hot, axis=1)

train_accuracy = np.mean(y_pred_train == y_true_train) * 100
test_accuracy = np.mean(y_pred_test == y_true_test) * 100

print(f"Train Accuracy: {train_accuracy:.2f}%")
print(f"Test Accuracy: {test_accuracy:.2f}%")


Epoch 0/1000, Loss: 0.6934
Epoch 100/1000, Loss: 0.3734
Epoch 200/1000, Loss: 0.3152
Epoch 300/1000, Loss: 0.2959
Epoch 400/1000, Loss: 0.2784
Epoch 500/1000, Loss: 0.2548
Epoch 600/1000, Loss: 0.2274
Epoch 700/1000, Loss: 0.2015
Epoch 800/1000, Loss: 0.1796
Epoch 900/1000, Loss: 0.1601
Train Accuracy: 95.45%
Test Accuracy: 83.61%
